**Binary Classification with a Deepl Learning Model**

In [ ]:
# packages
import os
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report, confusion_matrix, precision_score, f1_score
from sklearn.model_selection import TimeSeriesSplit, KFold
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf

from keras.models import Model
from keras.layers import Input, Conv1D, BatchNormalization, ReLU, GlobalAveragePooling1D, Dense
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical


In [ ]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load lagged data

In [ ]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/lagged_curtailment_target_features.csv', sep = ';', index_col=0)

In [ ]:
# convert index to datetime type
df.index = pd.to_datetime(df.index)
# drop wind speed to reduce multicollinearity
df.drop(['wind_speed_m/s', 'wind_speed_m/s_lag1', 'wind_speed_m/s_lag2', 'wind_speed_m/s_lag3'], inplace=True, axis=1)

Adding weekday and season cols

In [ ]:
'''
# Assuming df_lagged is your DataFrame with a datetime index
# Create a new column for weekday/weekend
df_lagged['weekday'] = df_lagged.index.weekday < 5

# Create a new column for season based on month
df_lagged['season'] = (df_lagged.index.month % 12 + 3) // 3

# Perform one-hot encoding for the 'season' column
df_lagged = pd.get_dummies(df_lagged, columns=['season'], drop_first=True)

# Perform one-hot encoding for the 'weekday' column
df_lagged = pd.get_dummies(df_lagged, columns=['weekday'], drop_first=True)

# Convert the 'weekday' and 'season' columns to numerical
df_lagged['weekday'] = df_lagged['weekday_True'].astype(int)
df_lagged['season'] = df_lagged[['season_2', 'season_3', 'season_4']].idxmax(axis=1).str.extract(r'(\d)').astype(int)

# Drop the intermediate columns created during one-hot encoding
df_lagged.drop(columns=['weekday_True', 'season_2', 'season_3', 'season_4'], inplace=True)

# Now df_lagged contains the original data with the new weekday and season columns encoded numerically
'''

Deep learning model with 3 hidden layers consisting of 100, 50 and 25 neurons

In [ ]:
# get desired df size
start_date = '2022-01-01'
end_date = '2023-12-31'
df = df.loc[start_date:end_date]

In [ ]:
# impute, scale pipeline and smote (for class imbalance)
preprocessor = Pipeline([
    ('scaler', StandardScaler())
])

smote = SMOTE(random_state=13)

# define features X and target y
X = df.drop(['redispatch', 'level'], axis = 1)
y = df['redispatch']

In [ ]:
# Define model architecture
def make_model(input_shape, num_classes):
    model = Sequential([
        InputLayer(input_shape),
        #Dense(100, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)), # add l2 regularization
        Dense(100, activation='relu'),
        Dense(50, activation='relu'),
        #tf.keras.layers.Dropout(0.5), #add dropout layers to reduce overfitting
        Dense(25, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Model parameters
input_shape = X.shape[1:]
num_classes = 1  # binary classification

# Define callbacks
callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
    EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

In [ ]:
# cross-testidation
n_splits = 50 #500
test_size = 48 #(48 - 12h with 15 min intervals)
tscv = TimeSeriesSplit(n_splits=n_splits, test_size=test_size, gap = 10)

# Initialize lists to store etestuation metrics for each fold
f1_scores_train = []
precision_scores_train = []
f1_scores_test = []
precision_scores_test = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
    print(f"Training on fold {fold}/{n_splits}")

    # Get the data for this fold
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    # preprocess train data
    X_train_balanced = preprocessor.fit_transform(X_train_fold)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train_balanced, y_train_fold)

    # Define validation and train data for this fold (but test seperate to keep it the same as other models)
    val_split = int(len(X_train_fold) * 0.2)  # Adjust validation split as needed
    X_val_fold, y_val_fold = X_train_fold[:val_split], y_train_fold[:val_split]
    X_train_fold, y_train_fold = X_train_fold[val_split:], y_train_fold[val_split:]

    # Create model instance
    model = make_model(input_shape, num_classes)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss="binary_crossentropy", metrics=["accuracy", tf.keras.metrics.Precision()])

    # to address the class imbalance further
    class_weights = ...

    # Train the model
    with tf.device('/device:GPU:0'):
      model.fit(
          X_train_fold, y_train_fold,
          validation_data=(X_val_fold, y_val_fold),
          epochs=100,
          batch_size=500,
          #class_weight=class_weights,
          callbacks=callbacks,
          verbose=1
      )

    # preprocess testidation/test data
    X_test_fold = preprocessor.transform(X_test_fold)

    # Predict on the test set
    y_pred_test = model.predict(X_test_fold)
    y_pred_test_classes = (y_pred_test > 0.5).astype(int)
    f1_test = f1_score(y_test_fold, y_pred_test_classes)
    precision_test = precision_score(y_test_fold, y_pred_test_classes)

    if precision_test != 0 and f1_test != 0:
        precision_scores_test.append(precision_test)
        f1_scores_test.append(f1_test)
    else:
        continue

    # Predict on the train set
    y_pred_train = model.predict(X_train_fold)
    y_pred_train_classes = (y_pred_train > 0.5).astype(int)
    f1_train = f1_score(y_train_fold, y_pred_train_classes)
    precision_train = precision_score(y_train_fold, y_pred_train_classes)
    f1_scores_train.append(f1_train)
    precision_scores_train.append(precision_train)

    # Print etestuation metrics for fold
    print(f"Fold {fold} - Test F1 score: {f1_test}, Test Precision: {precision_test}")
    print(f"Fold {fold} - Train F1 score: {f1_train}, Train Precision: {precision_train}")

# Print average scores across all folds
print("Average F1 score (test):", np.mean(f1_scores_test))
print("Average Precision score (test):", np.mean(precision_scores_test))
print("Average F1 score (Train):", np.mean(f1_scores_train))
print("Average Precision score (Train):", np.mean(precision_scores_train))


Training on fold 1/50
Epoch 1/100
109/109 [==============================] - 2s 6ms/step - loss: 118.1387 - accuracy: 0.7730 - precision_7: 0.1618 - val_loss: 11.0445 - val_accuracy: 0.8634 - val_precision_7: 0.0218 - lr: 0.0010
Epoch 2/100
109/109 [==============================] - 0s 4ms/step - loss: 40.0189 - accuracy: 0.7809 - precision_7: 0.1928 - val_loss: 14.7693 - val_accuracy: 0.9397 - val_precision_7: 0.0043 - lr: 0.0010
Epoch 3/100
109/109 [==============================] - 0s 4ms/step - loss: 42.0509 - accuracy: 0.7855 - precision_7: 0.1972 - val_loss: 12.6697 - val_accuracy: 0.9391 - val_precision_7: 0.0126 - lr: 0.0010
Epoch 4/100
109/109 [==============================] - 0s 4ms/step - loss: 46.5257 - accuracy: 0.7889 - precision_7: 0.2064 - val_loss: 134.6392 - val_accuracy: 0.4428 - val_precision_7: 0.0382 - lr: 0.0010
Epoch 5/100
109/109 [==============================] - 0s 4ms/step - loss: 57.1776 - accuracy: 0.7873 - precision_7: 0.2155 - val_loss: 27.9500 - val_ac

2/2 [==============================] - 0s 5ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training on fold 2/50
Epoch 1/100
109/109 [==============================] - 2s 6ms/step - loss: 285.1668 - accuracy: 0.7510 - precision_8: 0.1466 - val_loss: 57.5443 - val_accuracy: 0.8898 - val_precision_8: 0.0162 - lr: 0.0010
Epoch 2/100
109/109 [==============================] - 0s 4ms/step - loss: 82.4272 - accuracy: 0.7582 - precision_8: 0.1657 - val_loss: 33.1272 - val_accuracy: 0.9630 - val_precision_8: 0.0000e+00 - lr: 0.0010
Epoch 3/100
109/109 [==============================] - 0s 5ms/step - loss: 81.2329 - accuracy: 0.7648 - precision_8: 0.1818 - val_loss: 108.6396 - val_accuracy: 0.4055 - val_precision_8: 0.0340 - lr: 0.0010
Epoch 4/100
109/109 [==============================] - 0s 4ms/step - loss: 62.2980 - accuracy: 0.7697 - precision_8: 0.1868 - val_loss: 13.4291 - val_accuracy: 0.9415 - val_precision_8: 0.0135 - lr: 0.0010
Epoch 5/100
109/109 [==============================] - 1s 5ms/step - loss: 61.9862 - accuracy: 0.7782 - precision_8: 0.2040 - val_loss: 52.3848 - va

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/100
112/112 [==============================] - 2s 6ms/step - loss: 245.0044 - accuracy: 0.7756 - precision_55: 0.1601 - val_loss: 16.9149 - val_accuracy: 0.8314 - val_precision_55: 0.0364 - lr: 0.0010
Epoch 2/100
112/112 [==============================] - 0s 4ms/step - loss: 78.1189 - accuracy: 0.7832 - precision_55: 0.1802 - val_loss: 26.0444 - val_accuracy: 0.5340 - val_precision_55: 0.0498 - lr: 0.0010
Epoch 3/100
112/112 [==============================] - 0s 4ms/step - loss: 87.3586 - accuracy: 0.7867 - precision_55: 0.1916 - val_loss: 17.0078 - val_accuracy: 0.7617 - val_precision_55: 0.0478 - lr: 0.0010
Epoch 4/100
112/112 [==============================] - 0s 4ms/step - loss: 77.6474 - accuracy: 0.7859 - precision_55: 0.1887 - val_loss: 21.3005 - val_accuracy: 0.8543 - val_precision_55: 0.0261 - lr: 0.0010
Epoch 5/100
112/112 [==============================] - 1s 4ms/step - loss: 104.4433 - accuracy: 0.7923 - precision_55: 0.2021 - val_loss: 33.8688 - val_accuracy: 0.928

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/100
112/112 [==============================] - 2s 7ms/step - loss: 152.3674 - accuracy: 0.7649 - precision_56: 0.1478 - val_loss: 33.9453 - val_accuracy: 0.8574 - val_precision_56: 0.0219 - lr: 0.0010
Epoch 2/100
112/112 [==============================] - 1s 5ms/step - loss: 100.9188 - accuracy: 0.7781 - precision_56: 0.1622 - val_loss: 59.2897 - val_accuracy: 0.6393 - val_precision_56: 0.0195 - lr: 0.0010
Epoch 3/100
112/112 [==============================] - 1s 4ms/step - loss: 73.7469 - accuracy: 0.7801 - precision_56: 0.1757 - val_loss: 98.2371 - val_accuracy: 0.9516 - val_precision_56: 0.0000e+00 - lr: 0.0010
Epoch 4/100
112/112 [==============================] - 0s 4ms/step - loss: 73.8268 - accuracy: 0.7820 - precision_56: 0.1820 - val_loss: 20.1906 - val_accuracy: 0.8558 - val_precision_56: 0.0314 - lr: 0.0010
Epoch 5/100
112/112 [==============================] - 1s 4ms/step - loss: 69.2603 - accuracy: 0.7882 - precision_56: 0.1905 - val_loss: 15.9836 - val_accuracy: 0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Saving the deep learning model

In [ ]:
# Save deep learning model
deep_learning_model.save('deep_learning_model.h5')

Trying a convolutional neural network for classification

In [ ]:
start_date = pd.Timestamp('2020-01-01')
end_date = pd.Timestamp('2021-12-31 23:45:00')

# Filter rows that are outside of the specified timeframe
df_filtered = df_lagged[(df_lagged.index > end_date)]

df_filtered.head()

,redispatch,wind_direction_degrees,wind_direction_degrees_lag1,wind_direction_degrees_lag2,wind_direction_degrees_lag3,radiation_global_J/m2,radiation_global_J/m2_lag1,radiation_global_J/m2_lag2,radiation_global_J/m2_lag3,air_temperature_K,...,residual_load_MWh,residual_load_MWh_lag1,residual_load_MWh_lag2,residual_load_MWh_lag3,pumped_storage_MWh,pumped_storage_MWh_lag1,pumped_storage_MWh_lag2,pumped_storage_MWh_lag3,weekday,season
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0.0,270.0,270.0,270.0,270.0,0.0,0.0,0.0,0.0,282.05,...,212.98,215.08,223.97,230.88,39.98,50.40,46.20,47.45,0,2
2022-01-01 00:15:00,0.0,265.0,270.0,270.0,270.0,0.0,0.0,0.0,0.0,282.10,...,209.07,212.98,215.08,223.97,39.67,39.98,50.40,46.20,0,2
2022-01-01 00:30:00,0.0,270.0,265.0,270.0,270.0,0.0,0.0,0.0,0.0,282.35,...,208.60,209.07,212.98,215.08,43.30,39.67,39.98,50.40,0,2
2022-01-01 00:45:00,0.0,270.0,270.0,265.0,270.0,0.0,0.0,0.0,0.0,282.55,...,199.50,208.60,209.07,212.98,46.33,43.30,39.67,39.98,0,2
2022-01-01 01:00:00,0.0,270.0,270.0,270.0,265.0,0.0,0.0,0.0,0.0,282.65,...,194.30,199.50,208.60,209.07,39.12,46.33,43.30,39.67,0,2


In [ ]:
# Determine the cutoff point for splitting the data into train and test sets based on time
train_time = '2021-12-31'  # Define your train time
test_time_start = '2022-12-01'
test_time_end = '2023-01-01'

# Split the data into train and test sets based on time
train = df_filtered[(df_filtered.index > train_time) & (df_filtered.index < test_time_start)]
test = df_filtered[(df_filtered.index >= test_time_start) & (df_filtered.index < test_time_end)]

# Prepare the data
X_train = train.drop(columns=['redispatch'])  # Extract features
y_train = train['redispatch']  # Extract target

X_test = test.drop(columns=['redispatch'])  # Extract features
y_test = test['redispatch']  # Extract target

In [ ]:
# Normalize or scale the features if necessary
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Balance the classes in the training data using SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

Using cross validation with CNN model

In [ ]:
# Reshape the input data to have a 3D shape
X_train_balanced_reshaped = np.expand_dims(X_train_balanced, axis=-1)
X_test_scaled_reshaped = np.expand_dims(X_test_scaled, axis=-1)

# Define the number of splits for time series cross-validation
n_splits = 5

# Define the TimeSeriesSplit cross-validation strategy
tscv = TimeSeriesSplit(n_splits=n_splits)

# Define model architecture
def make_model(input_shape, num_classes):
    input_layer = Input(input_shape)

    conv1 = Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)

    conv2 = Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = ReLU()(conv2)

    conv3 = Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = BatchNormalization()(conv3)
    conv3 = ReLU()(conv3)

    gap = GlobalAveragePooling1D()(conv3)

    output_layer = Dense(num_classes, activation="softmax")(gap)

    return Model(inputs=input_layer, outputs=output_layer)

# Model parameters
input_shape = X_train_balanced_reshaped.shape[1:]
num_classes = len(np.unique(y_train_balanced))

# Define callbacks
callbacks = [
    ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss"),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001),
    EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

# Initialize lists to store evaluation metrics for each fold
f1_scores = []
precision_scores = []
train_f1_scores = []
train_precision_scores = []
test_f1_scores = []
test_precision_scores = []

# Iterate over each fold
for fold, (train_index, test_index) in enumerate(tscv.split(X_train_balanced_reshaped), 1):
    print(f"Training on fold {fold}/{n_splits}")

    # Get the data for this fold
    X_train_fold, X_val_fold = X_train_balanced_reshaped[train_index], X_train_balanced_reshaped[test_index]
    y_train_fold, y_val_fold = y_train_balanced[train_index], y_train_balanced[test_index]

    # Create model instance
    model = make_model(input_shape, num_classes)

    # Compile the model
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

    # Train the model
    history = model.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        epochs=500,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )

    # Predict on the validation set
    y_pred_val = model.predict(X_val_fold)
    y_pred_val_classes = np.argmax(y_pred_val, axis=1)

    # Calculate evaluation metrics for validation set
    f1 = f1_score(y_val_fold, y_pred_val_classes, average='macro')
    precision = precision_score(y_val_fold, y_pred_val_classes, average='macro')

    f1_scores.append(f1)
    precision_scores.append(precision)

    # Predict on the training set
    y_pred_train = model.predict(X_train_fold)
    y_pred_train_classes = np.argmax(y_pred_train, axis=1)

    # Calculate evaluation metrics for training set
    train_f1 = f1_score(y_train_fold, y_pred_train_classes, average='macro')
    train_precision = precision_score(y_train_fold, y_pred_train_classes, average='macro')

    train_f1_scores.append(train_f1)
    train_precision_scores.append(train_precision)

    # Predict on the test set
    y_pred_test = model.predict(X_test_scaled_reshaped)
    y_pred_test_classes = np.argmax(y_pred_test, axis=1)

    # Calculate evaluation metrics for test set
    test_f1 = f1_score(y_test, y_pred_test_classes, average='macro')
    test_precision = precision_score(y_test, y_pred_test_classes, average='macro')

    test_f1_scores.append(test_f1)
    test_precision_scores.append(test_precision)

# Print average scores across all folds
print("Average F1 score (validation):", np.mean(f1_scores))
print("Average precision score (validation):", np.mean(precision_scores))
print("Average F1 score (train):", np.mean(train_f1_scores))
print("Average precision score (train):", np.mean(train_precision_scores))
print("Average F1 score (test):", np.mean(test_f1_scores))
print("Average precision score (test):", np.mean(test_precision_scores))

Training on fold 1/5
Epoch 1/500
280/280 [==============================] - 6s 7ms/step - loss: 0.1080 - accuracy: 0.9729 - val_loss: 0.6433 - val_accuracy: 0.8777 - lr: 0.0010
Epoch 2/500
280/280 [==============================] - 2s 6ms/step - loss: 0.0575 - accuracy: 0.9860 - val_loss: 0.6115 - val_accuracy: 0.8777 - lr: 0.0010
Epoch 3/500
280/280 [==============================] - 2s 6ms/step - loss: 0.0508 - accuracy: 0.9860 - val_loss: 0.6887 - val_accuracy: 0.8777 - lr: 0.0010
Epoch 4/500
280/280 [==============================] - 2s 6ms/step - loss: 0.0469 - accuracy: 0.9862 - val_loss: 1.0179 - val_accuracy: 0.8777 - lr: 0.0010
Epoch 5/500
280/280 [==============================] - 2s 6ms/step - loss: 0.0416 - accuracy: 0.9865 - val_loss: 0.9479 - val_accuracy: 0.8777 - lr: 0.0010
Epoch 6/500
280/280 [==============================] - 2s 6ms/step - loss: 0.0383 - accuracy: 0.9873 - val_loss: 0.7310 - val_accuracy: 0.8773 - lr: 0.0010
Epoch 7/500
280/280 [======================

In [ ]:
#CNN result
#Average F1 score: 0.5076440234099121
#Average precision score: 0.6114438993430864

In [ ]:
# is it overfitting?
# Batchnormalization: method to make artificial NNs faster and more stable through re-centering and re-scaling layers inputs
# Dropout: regularisation technique for reducing overfitting in ANNs by preventing complex co-adaptations on training data. Performing model averaging


In [ ]:
'''
## safe
!mkdir -p saved_model
large_model.save('saved_model/my_large_model')

!zip -r saved_model.zip saved_model

# push to GitHub see [3]

## load
# unzip files
import zipfile
with zipfile.ZipFile(‘./saved_model.zip', 'r') as zip_ref:
    zip_ref.extractall('./')

# Load the saved model
with tf.device('/device:GPU:0'):
    new_large_model = tf.keras.models.load_model('saved_model/my_large_model')

# Check its architecture
new_large_model.summary()'''

"\n## safe \n!mkdir -p saved_model\nlarge_model.save('saved_model/my_large_model')\n\n!zip -r saved_model.zip saved_model\n\n# push to GitHub see [3]\n\n## load\n# unzip files \nimport zipfile\nwith zipfile.ZipFile(‘./saved_model.zip', 'r') as zip_ref:\n    zip_ref.extractall('./')\n\n# Load the saved model\nwith tf.device('/device:GPU:0'):\n    new_large_model = tf.keras.models.load_model('saved_model/my_large_model')\n\n# Check its architecture\nnew_large_model.summary()"